In [4]:
import numpy as np
from cvxopt import matrix, solvers
import time
def sdp(X,y,z,gamma):
    time1 = time.time()
    (m,n) = X.shape
    Y = np.concatenate([X, np.ones((m,1))], axis = 1)
    A11 = np.dot(Y.T, Y) #4*4
    A11 = (A11.T+A11)/2 #4*4
    A13 = np.dot(-Y.T, y).reshape(-1,1) #4*1
    A12 = np.dot(Y.T, z-y).reshape(-1,1) #4*1
    A22 = np.array([np.linalg.norm(z-y)**2]).reshape(1,1) #1*1
    A23 = -1*np.array([np.dot((z-y).T, y)]).reshape(1,1) #1*1
    A33 = np.dot(y.T, y) #1*1
    A1 = np.concatenate([A11, A12, A13], axis = 1)
    A2 = np.concatenate([A12.T, A22, A23], axis = 1)
    A3 = np.concatenate([A13.T, A23, A33], axis = 1)
    A = np.concatenate([A1, A2, A3], axis = 0)
   
    B = np.zeros((n+3, n+3))
    for i in range(n+1, n+3):
        for j in range(n+1, n+3):
            B[i][j]=1
    C = np.zeros((n+3, n+3))
    temp = np.identity(n+1)
    for i in range(n+1):
        for j in range(n+1):
            C[i][j]=1/gamma*temp[i][j]
    for i in range(n+1, n+3):
        for j in range(n+1, n+3):
            C[i][j]=-0.5
    B = B.reshape(1,-1)
    C = -1 * C.reshape(1,-1)
   
    h = [matrix(A.tolist())]
    c = matrix([1.,0.])
    sol = solvers.sdp(c, Gs=[matrix([B.tolist()[0], C.tolist()[0]])], hs=h)
    time2 = time.time()
    print(sol['x'])
    print(time2-time1)
    
    
    
    
    




In [5]:
import random
import numpy as np
def getTime_realData(X, y, z, gamma,size_range):
    fold = 10
    num_size = len(size_range)
    random.seed(2021)
    (m,n) = X.shape
    ssdp_yes = 1
    time_ssdp = np.zeros(num_size)
    time_ssdp_std = np.zeros(num_size)
    optval_ssdp = np.zeros(num_size)
    w_ssdp = np.zeros((n+1,fold))
    for i in range(num_size):
        print('Starting a dataset size of ' + str(i));
        timessdp = np.zeros(fold)
        optvalssdp = np.zeros(fold)
        m_curr = size_range[i]
        for idx in range(fold):
            ridx = random.sample(range(m), m_curr)
            if ssdp_yes:
                print('Doing ssdp')
                timessdp[idx] = sdp(X[ridx] , y[ridx], z[ridx], gamma);  
        time_ssdp[i] = np.mean(timessdp)
        time_ssdp_std[i]=np.std(timessdp)
    return (time_ssdp, time_ssdp_std)

            
       
           
    
    

In [6]:
from matplotlib.pyplot import table
import pandas as pd
from sklearn.preprocessing import normalize
dataname_list = ["wine_modest", "wine_severe", "insurance_modest","insurance_severe"]
len_name = len(dataname_list)
gamma_list = [1e-1]
normalize_yes = 1

for gamma_idx in range(len(gamma_list)):
    gamma = gamma_list[gamma_idx]
    ssdp_mr = np.zeros((len_name,1))
    ssdp_std = np.zeros((len_name,1))
    for idx in range(len_name):
        dataname = dataname_list[idx]
        [X, y, z, const, gamma_list, gamma_time, datasize_list] = data_read(dataname)
       
        if normalize_yes:
            X = normalize(X, axis=1, norm='l1')
        
        Infossdp = getTime_realData(X, y, z, gamma, datasize_list);
        ssdp_mr  = Infossdp[0]
        ssdp_std  = Infossdp[1]

        print(ssdp_mr, ssdp_std)
        df = pd.DataFrame({'datasize': datasize_list, 'ssdp_mr': ssdp_mr, 'ssdp_std': ssdp_std})
         
        table_name_cmp ='result/'+str(dataname)+'_time.csv'
        df.to_csv(table_name_cmp)


   

NameError: name 'data_read' is not defined